# Webscraping #

## Cavin Jacobson ##

### In this exercise, you'll practice using BeautifulSoup to parse the content of a web page. The page that you'll be scraping, https://realpython.github.io/fake-jobs/, contains job listings. Your job is to extract the data on each job and convert into a pandas DataFrame. ###

In [4]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
from IPython.core.display import HTML
import pandas as pd
import io
import re

**1. Start by performing a GET request on the url above and convert the response into a BeautifulSoup object.**

In [6]:
URL = 'https://realpython.github.io/fake-jobs/'

response = requests.get(URL)


In [7]:
job_soup = BeautifulSoup(response.text, features='html.parser')

In [8]:
print(job_soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Fake Python
  </title>
  <link href="https://cdn.jsdelivr.net/npm/bulma@0.9.2/css/bulma.min.css" rel="stylesheet"/>
 </head>
 <body>
  <section class="section">
   <div class="container mb-5">
    <h1 class="title is-1">
     Fake Python
    </h1>
    <p class="subtitle is-3">
     Fake Jobs for Your Web Scraping Journey
    </p>
   </div>
   <div class="container">
    <div class="columns is-multiline" id="ResultsContainer">
     <div class="column is-half">
      <div class="card">
       <div class="card-content">
        <div class="media">
         <div class="media-left">
          <figure class="image is-48x48">
           <img alt="Real Python Logo" src="https://files.realpython.com/media/real-python-logo-thumbnail.7f0db70c2ed2.jpg?__no_cf_polish=1"/>
          </figure>
         </div>
         <div class="media-content">
          <h2 c

**1a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.**

In [10]:
job_soup.find('h2').text

'Senior Python Developer'

**1b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.**

In [12]:
job_list = [
    job.text for job in job_soup.findAll('h2')
]

**1c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end.**

In [14]:
job_place = (
    [job
     .text
     .replace('\n', '') 
     for job in job_soup
     .findAll('h3')]
)

In [15]:
job_time = (
    [job
     .text
     .replace('\n', '') 
     for job in job_soup
     .findAll(class_= 'is-small has-text-grey')]
)

In [16]:
job_loc = (
    [job
     .text
     .replace('\n', '')
     .replace('        ','')
     .replace('      ','')
     for job in job_soup
     .findAll(class_= 'location')]
)

**1d. Take the lists that you have created and combine them into a pandas DataFrame.**

In [18]:
job_info = (
    pd
    .DataFrame(list(zip(job_list, job_time, job_loc, job_place)))
    .rename(columns = {0: 'Title', 1: 'date', 2: 'location', 3: 'company'})
)

**2. Next, add a column that contains the url for the "Apply" button. Try this in two ways.**

**2a. First, use the BeautifulSoup find_all method to extract the urls.**

In [21]:
urls = []

for link in job_soup.findAll("a", href=True)[1::2]:
    urls.append(link['href'])

In [22]:
job_info['URLS'] = urls

In [23]:
job_info

Title        date              location  \
0              Senior Python Developer  2021-04-08       Stewartbury, AA   
1                      Energy engineer  2021-04-08  Christopherville, AA   
2                      Legal executive  2021-04-08   Port Ericaburgh, AA   
3               Fitness centre manager  2021-04-08     East Seanview, AP   
4                      Product manager  2021-04-08   North Jamieview, AP   
..                                 ...         ...                   ...   
95  Museum/gallery exhibitions officer  2021-04-08      Lake Abigail, AE   
96            Radiographer, diagnostic  2021-04-08        Jacobshire, AP   
97              Database administrator  2021-04-08        Port Susan, AE   
98                  Furniture designer  2021-04-08     North Tiffany, AA   
99                         Ship broker  2021-04-08     Michelleville, AP   

                       company  \
0     Payne, Roberts and Davis   
1             Vasquez-Davidson   
2   Jackson, Chambers and Levy   
3               Savage-Bradley   
4                  Ramirez Inc   
..                         ...   
95     Nguyen, Yoder and Petty   
96                  Holder LLC   
97              Yates-Ferguson   
98             Ortega-Lawrence   
99   Fuentes, Walls and Castro   

                                                 URLS  
0   https://realpython.github.io/fake-jobs/jobs/se...  
1   https://realpython.github.io/fake-jobs/jobs/en...  
2   https://realpython.github.io/fake-jobs/jobs/le...  
3   https://realpython.github.io/fake-jobs/jobs/fi...  
4   https://realpython.github.io/fake-jobs/jobs/pr...  
..                                                ...  
95  https://realpython.github.io/fake-jobs/jobs/mu...  
96  https://realpython.github.io/fake-jobs/jobs/ra...  
97  https://realpython.github.io/fake-jobs/jobs/da...  
98  https://realpython.github.io/fake-jobs/jobs/fu...  
99  https://realpython.github.io/fake-jobs/jobs/sh...  

[100 rows x 5 columns]

**2b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.**

In [25]:
#title = 

#URL = f'https://realpython.github.io/fake-jobs/jobs/{title}.html

**3. Finally, we want to get the job description text for each job.**

**3a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph.**

In [28]:
response = requests.get('https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html')

soup = BeautifulSoup(response.text, features='html.parser')

soup.findAll('div')[1].find('p').text

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

**3b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along."**

In [48]:
def url_prompt(url):
    # Allows for user to input a url from https://realpython.github.io/fake-jobs/  
    # and retrieve the job description.
    # The url must be input using [] 
    for URL in url:
        response = requests.get(URL)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, features='html.parser')
        p1 = soup.findAll('div')[1].find('p').text
    return print(p1)

In [108]:
url_prompt(['https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html'])

At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.


In [73]:
description_list = []

for URL in urls:
    response = requests.get(URL)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, features='html.parser')
    p1 = soup.findAll('div')[1].find('p').text
    description_list.append(p1)

**3c. Use the .apply method on the url column you created above to retrieve the description text for all of the jobs.**

In [95]:
urls = []

def url_get(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features='html.parser')
    for links in soup.findAll("a", href=True)[1::2]:
        urls.append(link['href'])
        for URL in urls:
            response = requests.get(URL)
            soup = BeautifulSoup(response.content, features='html.parser')
            p1 = soup.findAll('div')[1].find('p').text
    return print(p1)

In [99]:
job_info['URLS'].apply(url_get('https://realpython.github.io/fake-jobs/'))

Management common popular project only. Must small hair strong reveal future girl. Mother anything western Congress. You thought PM charge or put upon. Least building military seem. Glass type structure so magazine worker. Message become of Republican life field game look.


TypeError: 'NoneType' object is not callable

In [104]:
job_info['Description'] = description_list

In [106]:
job_info

Title        date              location  \
0              Senior Python Developer  2021-04-08       Stewartbury, AA   
1                      Energy engineer  2021-04-08  Christopherville, AA   
2                      Legal executive  2021-04-08   Port Ericaburgh, AA   
3               Fitness centre manager  2021-04-08     East Seanview, AP   
4                      Product manager  2021-04-08   North Jamieview, AP   
..                                 ...         ...                   ...   
95  Museum/gallery exhibitions officer  2021-04-08      Lake Abigail, AE   
96            Radiographer, diagnostic  2021-04-08        Jacobshire, AP   
97              Database administrator  2021-04-08        Port Susan, AE   
98                  Furniture designer  2021-04-08     North Tiffany, AA   
99                         Ship broker  2021-04-08     Michelleville, AP   

                       company  \
0     Payne, Roberts and Davis   
1             Vasquez-Davidson   
2   Jackson, Chambers and Levy   
3               Savage-Bradley   
4                  Ramirez Inc   
..                         ...   
95     Nguyen, Yoder and Petty   
96                  Holder LLC   
97              Yates-Ferguson   
98             Ortega-Lawrence   
99   Fuentes, Walls and Castro   

                                                 URLS  \
0   https://realpython.github.io/fake-jobs/jobs/se...   
1   https://realpython.github.io/fake-jobs/jobs/en...   
2   https://realpython.github.io/fake-jobs/jobs/le...   
3   https://realpython.github.io/fake-jobs/jobs/fi...   
4   https://realpython.github.io/fake-jobs/jobs/pr...   
..                                                ...   
95  https://realpython.github.io/fake-jobs/jobs/mu...   
96  https://realpython.github.io/fake-jobs/jobs/ra...   
97  https://realpython.github.io/fake-jobs/jobs/da...   
98  https://realpython.github.io/fake-jobs/jobs/fu...   
99  https://realpython.github.io/fake-jobs/jobs/sh...   

                                          Description  
0   Professional asset web application environment...  
1   Party prevent live. Quickly candidate change a...  
2   Administration even relate head color. Staff b...  
3   Tv program actually race tonight themselves tr...  
4   Traditional page a although for study anyone. ...  
..                                                ...  
95  Paper age physical current note. There reality...  
96  Able such right culture. Wrong pick structure ...  
97  Create day party decade high clear. Past trade...  
98  Pressure under rock next week. Recognize so re...  
99  Management common popular project only. Must s...  

[100 rows x 6 columns]